In [2]:
import clip
import torch
import pandas as pd
import numpy as np
from PIL import Image

In [3]:
def filter_df(df, race=None, gender=None):
    new_df = df
    if gender:
        new_df = new_df[new_df['gender'] == gender]
    if race:
        new_df = new_df[new_df['race'] == race]
    return new_df

In [4]:
%matplotlib inline

print('\nLoading model...')
available_models = ['RN50', 'RN101', 'RN50x4', 'RN50x16']
layers = ['layer4', 'layer3', 'layer2', 'layer1']

clip_model = available_models[0]
saliency_layer = layers[0]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(clip_model, device=device, jit=False)
print(f"Done! Model loaded to {device} device")


Loading model...
Done! Model loaded to cuda device


In [5]:
path = "/home/lazye/Documents/ufrgs/mcs/datasets/FairFace/"
fface_df = pd.read_csv(f"{path}/train/fairface_label_train.csv")

In [6]:
man_df = filter_df(fface_df, 'White', 'Male')
woman_df = filter_df(fface_df, 'White', 'Female')

In [9]:
man_df['gender'].value_counts()

gender
Male    8701
Name: count, dtype: int64

In [7]:
def generate_embeddings(df, n, out):
    for file in df.iloc[0:n]:
        img_path = path + file
        img = Image.open(img_path)
        img_input = preprocess(img).unsqueeze(0).to(device)

        with torch.no_grad():
            image_features = model.encode_image(img_input)
        
        image_features /= image_features.norm(dim=-1, keepdim=True)
        file_num = file.split('/')[-1].split('.')[0]
        np.save(f'{out}/{file_num}.npy', image_features.cpu().numpy())

In [17]:
generate_embeddings(woman_df['file'], 1000, 'woman_embeddings')